## Data Science Job Postings with Salaries (2025)
### Scraped Data Science Job Postings with Salaries for ML & Career Analytics (2025)
#### Data may be found here: https://www.kaggle.com/datasets/elahehgolrokh/data-science-job-postings-with-salaries-2025

In [31]:
# import libraries needed for EDA

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

# Core data handling
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import bokeh.plotting as bkp
import altair as alt


# Statistics
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf


In [35]:
# load data into dataframe

df = pd.read_csv("data_science_job_posts_2025.csv")

In [37]:
# examine first 10 rows

df.head(10)

,job_title,seniority_level,status,company,location,post_date,headquarter,industry,ownership,company_size,revenue,salary,skills
0,data scientist,senior,hybrid,company_003,"Grapevine, TX . Hybrid",17 days ago,"Bentonville, AR, US",Retail,Public,€352.44B,Public,"€100,472 - €200,938","['spark', 'r', 'python', 'scala', 'machine lea..."
1,data scientist,lead,hybrid,company_005,"Fort Worth, TX . Hybrid",15 days ago,"Detroit, MI, US",Manufacturing,Public,"155,030",€51.10B,"€118,733","['spark', 'r', 'python', 'sql', 'machine learn..."
2,data scientist,senior,on-site,company_007,"Austin, TX . Toronto, Ontario, Canada . Kirkla...",a month ago,"Redwood City, CA, US",Technology,Public,"25,930",€33.80B,"€94,987 - €159,559","['aws', 'git', 'python', 'docker', 'sql', 'mac..."
3,data scientist,senior,hybrid,company_008,"Chicago, IL . Scottsdale, AZ . Austin, TX . Hy...",8 days ago,"San Jose, CA, US",Technology,Public,"34,690",€81.71B,"€112,797 - €194,402","['sql', 'r', 'python']"
4,data scientist,NaN,on-site,company_009,On-site,3 days ago,"Stamford, CT, US",Finance,Private,"1,800",Private,"€114,172 - €228,337",[]
5,data scientist,lead,NaN,company_013,"New York, NY",3 months ago,"New York, NY, US",Technology,Private,150,€2.16B,"€196,371 - €251,170","['scikit-learn', 'python', 'scala', 'sql', 'ma..."
6,data scientist,junior,NaN,company_014,"Berkeley, CA",15 days ago,"Berkeley, CA, US",Education,NaN,"17,471",Education,"€51,330 - €70,144",[]
7,machine learning engineer,senior,on-site,company_015,"Menlo Park, CA",9 days ago,"Menlo Park, CA, US",Technology,Public,900,Public,"€121,480 - €132,440",['machine learning']
8,data scientist,senior,remote,company_019,Fully Remote,6 days ago,"Boston, MA, US",Technology,Private,126,Private,"€207,331",[]
9,data scientist,senior,on-site,company_021,On-site,a day ago,"San Francisco, CA, US",Technology,Public,"5,520",€61.06B,"€219,201",[]


In [39]:
df.shape

(944, 13)

In [41]:
# List all columns and their data types

df.dtypes.reset_index().rename(columns={'index': 'Column', 0: 'DataType'})

,Column,DataType
0,job_title,object
1,seniority_level,object
2,status,object
3,company,object
4,location,object
5,post_date,object
6,headquarter,object
7,industry,object
8,ownership,object
9,company_size,object


### The 'location' column can be dropped, as it contains the info already in 'status' and repeats the country info in 'headquarter', which we will rename 'headquarters'.

In [46]:
# Drop 'location' column
df = df.drop(columns=["location"])

# Rename 'headquarter' to 'headquarters'
df = df.rename(columns={"headquarter": "headquarters"})

In [48]:
# Count null values per column

null_counts = df.isnull().sum().reset_index()
null_counts.columns = ["Column", "NullValues"]
null_counts


,Column,NullValues
0,job_title,3
1,seniority_level,60
2,status,256
3,company,0
4,post_date,0
5,headquarters,0
6,industry,0
7,ownership,47
8,company_size,0
9,revenue,15


### We will drop the null values in 'job_title' and 'revenue' and change null values in 'seniority_level', 'status' and 'ownership to 'unknown'.

In [62]:
# Drop rows where 'job_title' or 'revenue' is null
df = df.dropna(subset=["job_title", "revenue"])

# Replace nulls in 'seniority_level', 'status', and 'ownership' with 'unknown'
df["seniority_level"] = df["seniority_level"].fillna("unknown")
df["status"] = df["status"].fillna("unknown")
df["ownership"] = df["ownership"].fillna("unknown")

In [64]:
# Count null values per column

null_counts = df.isnull().sum().reset_index()
null_counts.columns = ["Column", "NullValues"]
null_counts


,Column,NullValues
0,job_title,0
1,seniority_level,0
2,status,0
3,company,0
4,post_date,0
5,headquarters,0
6,industry,0
7,ownership,0
8,company_size,0
9,revenue,0


### We will now have the 'headquarters' column show only the country code for the location.

In [75]:
# Keep only the last part after the last comma in 'headquarters' (assumed to be the country)
df["headquarters"] = df["headquarters"].apply(lambda x: str(x).split(",")[-1].strip())

# Preview unique values to confirm transformation
df["headquarters"].unique()[:20]


array(['US', 'CA', 'SE', 'CH', 'IN', 'GB', 'DK', 'DE', 'TW', 'NL', 'AU',
       'EE', 'FR', 'SG', 'IT', 'BR', 'AT', 'JP', 'IE', 'ES'], dtype=object)

### Let's convert the country codes into full names:

In [78]:
# Dictionary mapping 2-letter codes to full country names
country_map = {
    "US": "United States",
    "CA": "Canada",
    "SE": "Sweden",
    "CH": "Switzerland",
    "IN": "India",
    "GB": "United Kingdom",
    "DK": "Denmark",
    "DE": "Germany",
    "TW": "Taiwan",   # force "Taiwan" instead of "Taiwan, Province of China"
    "NL": "Netherlands",
    "AU": "Australia",
    "EE": "Estonia",
    "FR": "France",
    "SG": "Singapore",
    "IT": "Italy",
    "BR": "Brazil",
    "AT": "Austria",
    "JP": "Japan",
    "IE": "Ireland",
    "ES": "Spain"
    # add more codes if needed
}

# Map headquarters column using dictionary
df["headquarters"] = df["headquarters"].map(country_map).fillna(df["headquarters"])

# Preview unique values
print(df["headquarters"].unique()[:20])


['United States' 'Canada' 'Sweden' 'Switzerland' 'India' 'United Kingdom'
 'Denmark' 'Germany' 'Taiwan' 'Netherlands' 'Australia' 'Estonia' 'France'
 'Singapore' 'Italy' 'Brazil' 'Austria' 'Japan' 'Ireland' 'Spain']


### The 'skills' column is problematic: Some cells contain many items, some are empty lists. We will replace the column's values with the number of items in the lists.

In [81]:
import ast

# Function to count items in list-like strings
def count_skills(value):
    try:
        skills_list = ast.literal_eval(value)  # safely evaluate string to Python list
        if isinstance(skills_list, list):
            return len(skills_list)
        return 0
    except:
        return 0  # if parsing fails, treat as empty

# Apply to 'skills' column
df["skills"] = df["skills"].apply(count_skills)

In [83]:
# List all columns and their data types

df.dtypes.reset_index().rename(columns={'index': 'Column', 0: 'DataType'})

,Column,DataType
0,job_title,object
1,seniority_level,object
2,status,object
3,company,object
4,post_date,object
5,headquarters,object
6,industry,object
7,ownership,object
8,company_size,object
9,revenue,object


### The 'salary' column is also problematic: It needs to be converted to numerical data, and if there is a range given we will use the mean of the range.

In [87]:
import re
import numpy as np
import pandas as pd

def normalize_salary_text(s: str) -> str:
    """Remove currency symbols/words and normalize 'k' shorthand."""
    if not isinstance(s, str):
        return ""
    t = s.strip()
    # Normalize dashes
    t = t.replace("–", "-").replace("—", "-").replace(" to ", "-")
    # Remove common currency symbols/words
    t = re.sub(r'(€|\$|£|\bEUR\b|\bUSD\b|\bGBP\b|per\s*year|/year|yr|/mo|per\s*month|monthly)', '', t, flags=re.IGNORECASE)
    # Expand 55k -> 55000
    def k_to_num(m):
        return str(float(m.group(1)) * 1000)
    t = re.sub(r'(\d+(?:\.\d+)?)\s*[kK]\b', k_to_num, t)
    # Remove commas and extra spaces
    t = t.replace(",", " ")
    t = re.sub(r'\s+', ' ', t).strip()
    return t

def parse_salary_to_number(s):
    """Return a float salary by taking the mean if a range is given, else single value. NaN if no numbers."""
    if pd.isna(s):
        return np.nan
    t = normalize_salary_text(s)
    # Extract numbers (allow decimals)
    nums = re.findall(r'\d+(?:\.\d+)?', t)
    if not nums:
        return np.nan
    vals = [float(x) for x in nums]
    # If more than 2 numbers appear (e.g., '50 000 - 70 000'), merge thousands that were spaced:
    # Heuristic: join adjacent numbers where one is 2-3 digits and the next is 3 digits to form a thousands number.
    if len(vals) > 2:
        # Try to reconstruct by joining pairs like [50,000] -> 50000
        tokens = re.findall(r'\d+(?:\.\d+)?', t)
        # Re-parse from original cleaned string by removing spaces within numbers
        t2 = re.sub(r'(\d)\s+(\d{3}\b)', r'\1\2', t)  # turn "50 000" into "50000"
        nums2 = re.findall(r'\d+(?:\.\d+)?', t2)
        vals = [float(x) for x in nums2]
    # Use mean if range (2+ numbers), else the single number
    if len(vals) >= 2:
        return float(np.mean([vals[0], vals[1]]))
    return float(vals[0])

# Apply conversion
original_non_null = df['salary'].notna().sum()
df['salary_num'] = df['salary'].apply(parse_salary_to_number)

# Report summary
summary = {
    "rows_total": len(df),
    "salary_non_null_before": int(original_non_null),
    "salary_parsed_non_null": int(df['salary_num'].notna().sum()),
    "salary_unparsed_nulls": int(df['salary_num'].isna().sum())
}

df['salary'] = df['salary_num']
df = df.drop(columns=['salary_num'])

summary, df['salary'].head(10), df['salary'].dtype


({'rows_total': 926,
  'salary_non_null_before': 926,
  'salary_parsed_non_null': 926,
  'salary_unparsed_nulls': 0},
 0    150705.0
 1       425.5
 2    127273.0
 3    153599.5
 4    171254.5
 5    223770.5
 6     60737.0
 7    126960.0
 8       269.0
 9       210.0
 Name: salary, dtype: float64,
 dtype('float64'))

In [89]:
# List all columns and their data types

df.dtypes.reset_index().rename(columns={'index': 'Column', 0: 'DataType'})

,Column,DataType
0,job_title,object
1,seniority_level,object
2,status,object
3,company,object
4,post_date,object
5,headquarters,object
6,industry,object
7,ownership,object
8,company_size,object
9,revenue,object


### The 'revenue' column contains some numerical data, but many cells which contain info like 'private' and 'education'. We will drop this column. Also, we will make sure company_size contains only numerical values.

In [92]:
# Drop 'revenue' column
df = df.drop(columns=["revenue"])

# Convert 'company_size' to numeric values
# Many company_size entries are ranges (e.g., "51-200") or open-ended ("10000+").
# We'll replace ranges with their midpoint, and "10000+" with 10000.

def clean_company_size(value):
    if pd.isna(value):
        return np.nan
    text = str(value).replace(",", "").strip()
    if "-" in text:  # range case
        parts = text.split("-")
        try:
            nums = [int(p) for p in parts if p.isdigit() or p.isnumeric()]
            if len(nums) == 2:
                return int(np.mean(nums))
        except:
            return np.nan
    if text.endswith("+"):  # e.g., "10000+"
        try:
            return int(text[:-1])
        except:
            return np.nan
    try:
        return int(text)
    except:
        return np.nan

df["company_size"] = df["company_size"].apply(clean_company_size)

# Verify transformation
df["company_size"].head(10), df["company_size"].dtype


(0         NaN
 1    155030.0
 2     25930.0
 3     34690.0
 4      1800.0
 5       150.0
 6     17471.0
 7       900.0
 8       126.0
 9      5520.0
 Name: company_size, dtype: float64,
 dtype('float64'))

### In company_size, we'll fill in any NaN values with the median of the column:

In [95]:
# Fill NaN values in 'company_size' with median
median_size = df["company_size"].median()
df["company_size"] = df["company_size"].fillna(median_size)

# Verify no nulls remain
df["company_size"].isnull().sum(), df["company_size"].head(10)

(0,
 0     20040.0
 1    155030.0
 2     25930.0
 3     34690.0
 4      1800.0
 5       150.0
 6     17471.0
 7       900.0
 8       126.0
 9      5520.0
 Name: company_size, dtype: float64)